In [31]:
loc_info = {0: ['안암역'], 1: ['노원역'], 2: ['한양대역'], 3: ['서울역']}

class settlement_algorithm():
    def __init__(self, loc_info):
        self.loc_info = loc_info

    def individual_move(self):
        loc_info = self.loc_info
        # global individual_distance, individual_duration, individual_cost, addr_info, passenger_routing
        self.individual_distance = [29321, 8867, 8932]
        self.individual_duration = [2955, 2197, 2530]
        self.individual_cost = [26700, 14300, 16000]
        self.passenger_routing = ['노원역', '한양대역', '안암역', '서울역']
        loc_info = {0: ['안암역', 127.02934916, 37.58631392], 1: ['노원역', 127.06042771, 37.65486207],  2: ['한양대역', 127.04368209, 37.55462342],  3: ['서울역', 126.96921669, 37.55320558]}
        self.addr_info = {loc_info[i][0] : {'no': i, 'lon': loc_info[i][1], 'lat': loc_info[i][2]} for i in대 range(len(loc_info))}
        print("동승 전 경로 visualization")

        return {'individual_distance':self.individual_distance, 'individual_duration':self.individual_duration, 'individual_cost':self.individual_cost}

    def routing_move(self):
        self.loc_info = loc_info
        # global routing_distance, routing_duration, routing_cost, delay_ratio
        self.routing_distance = [28761, 13989, 8932]
        self.routing_duration = [5347, 3382, 2530]
        self.routing_cost = [34500, 20100, 16000]
        self.delay_ratio = [0.49379932356257045, 0.34216255442670535, 0.0]
        print("동승 후 경로 visualization")

        return {"routing_distance":self.routing_distance, "routing_duration":self.routing_duration, "routing_cost":self.routing_cost, "delay_ratio": self.delay_ratio, 'passenger_routing': self.passenger_routing}

    def settlement_optimizer(self, eplison = 0.03, gamma = 0.05, LB = 0.3, UB = 0.5):
        import gurobipy as gp
        from gurobipy import GRB
        T = self.routing_cost[0]
        eplison = eplison
        gamma = gamma
        LB = LB
        UB = UB


        I1, I2, I3 = self.individual_cost
        R1, R2, R3 = self.delay_ratio

        ## matching ALGorithm
        try:
          # Create a new model
          m = gp.Model('cost_optimization1')
          m.Params.LogToConsole = 0

          # Create variables
          alpha = m.addVar(vtype=GRB.CONTINUOUS, name='alpha')
          beta = m.addVar(vtype=GRB.CONTINUOUS, name='beta')

          # Set objective
          m.setObjective(I1*(1-(alpha+beta*R1))+I2*(1-(alpha+beta*R2))+I3*(1-(alpha+beta*R3))-T, GRB.MINIMIZE)

          # Add constraint
          m.addConstr(max(R1,R2,R3)*beta-UB*alpha <= 0,'const_1')
          m.addConstr(max(R1,R2,R3)*beta-LB*alpha >= 0,'const_2')
          m.addConstr(I1*(1-(alpha+beta*R1))+I2*(1-(alpha+beta*R2))+I3*(1-(alpha+beta*R3))-T >= 0,'const_3')

          m.addConstr(alpha <= 0.5)
          m.addConstr(alpha >= 0)
          m.addConstr(beta <= 0.5)
          m.addConstr(beta >= 0)
          #m.addConstr(gamma <= 1)

          # Optimize model
          m.optimize()

          for v in m.getVars():
            # print('%s : %g' % (v.varName, v.x))
            globals()[v.varName] = v.x
          # print('Obj : %g' % m.objVal)

        except gp.GurobiError as e:
          print('Error code ' + str(e.errno) + ': ' + str(e))

        print(3)

        return alpha, beta

In [32]:
settlement = settlement_algorithm(loc_info)
individual_move = settlement.individual_move()
print()
print(individual_move)
print()
routing_move = settlement.routing_move()
print()
print(routing_move)
print()
optimizer = settlement.settlement_optimizer()
print()
print(optimizer)

동승 전 경로 visualization

{'individual_distance': [29321, 8867, 8932], 'individual_duration': [2955, 2197, 2530], 'individual_cost': [26700, 14300, 16000]}

동승 후 경로 visualization

{'routing_distance': [28761, 13989, 8932], 'routing_duration': [5347, 3382, 2530], 'routing_cost': [34500, 20100, 16000], 'delay_ratio': [0.49379932356257045, 0.34216255442670535, 0.0], 'passenger_routing': ['노원역', '한양대역', '안암역', '서울역']}

3

(<gurobi.Var *Awaiting Model Update*>, <gurobi.Var *Awaiting Model Update*>)


In [33]:
individual_distance = individual_move['individual_distance']
individual_duration = individual_move['individual_duration']
individual_cost = individual_move['individual_cost']

routing_distance = routing_move['routing_distance']
routing_duration = routing_move['routing_duration']
routing_cost = routing_move['routing_cost']
delay_ratio = routing_move['delay_ratio']

I1, I2, I3 = individual_cost
R1, R2, R3 = delay_ratio

print(f'A {alpha*100:.2f}+{beta*R1*100:.2f} => {(alpha+beta*R1)*100:.2f}% 할인')
print(f'B {alpha*100:.2f}+{beta*R2*100:.2f} => {(alpha+beta*R2)*100:.2f}% 할인')
print(f'C {alpha*100:.2f}+{beta*R3*100:.2f} => {(alpha+beta*R3)*100:.2f}% 할인')
print()
print(f'A {I1} 원 >>> {int((1-alpha+beta*R1)*I1)} 원')
print(f'B {I2} 원 >>> {int((1-alpha+beta*R2)*I2)} 원')
print(f'C {I3} 원 >>> {int((1-alpha+beta*R3)*I3)} 원')

routing_individual_cost = [int((1-alpha+beta*R1)*I1),int((1-alpha+beta*R2)*I2),int((1-alpha+beta*R3)*I3)]

A 33.10+9.93 => 43.03% 할인
B 33.10+6.88 => 39.98% 할인
C 33.10+0.00 => 33.10% 할인

A 26700 원 >>> 20514 원
B 14300 원 >>> 10551 원
C 16000 원 >>> 10704 원


In [34]:
print('혼자탔을 때')
print(f'합계 : {sum(individual_cost)} 원')
print([f'{rou} 원' for rou in individual_cost])
print([f'{rou/1000:.2f} km' for rou in individual_distance])
print([f'{rou/60:.2f} 분' for rou in individual_duration])
print()
print('동승시')
print(f'합계 : {routing_cost[0]} 원')
print([f'{rou} 원' for rou in routing_individual_cost])
print([f'{rou/1000:.2f} km' for rou in routing_distance])
print([f'{rou/60:.2f} 분' for rou in routing_duration])

혼자탔을 때
합계 : 57000 원
['26700 원', '14300 원', '16000 원']
['29.32 km', '8.87 km', '8.93 km']
['49.25 분', '36.62 분', '42.17 분']

동승시
합계 : 34500 원
['20514 원', '10551 원', '10704 원']
['28.76 km', '13.99 km', '8.93 km']
['89.12 분', '56.37 분', '42.17 분']
